# NbConvert

## Command line usage

`NbConvert` is the library, and the command line tool that allow to convert from notebook to other formats.
It is a technological preview in 1.0 but is already usable and highly configurable.
It ships already with many default available formats : `html`, `latex`, `markdown`, `python`, `rst` and  `slides`
which are fully base on Jinja templating engine, so writing a converter for your custom format or tweeking the existing 
one should be extra simple.

You can invoke nbconvert by doing

```bash
$ ipython nbconvert <options and arguments>
```

Call `ipython nbconvert` with the `--help` flag or no aruments to get basic help on how to use it.
For more information about configuration use the `--help-all` flag

### Basic export

We will be converting `Custom Display Logic.ipynb`. 
Be sure to have runed some of the cells in it to have output otherwise you will only see input in nbconvert.
Nbconvert **do not execute the code** in the notebook files, it only converts what is inside.

In [1]:
%%bash
ipython nbconvert  '04 - Custom Display Logic.ipynb'

[NbConvertApp] Using existing profile dir: u'/Users/bussonniermatthias/.ipython/profile_default'
[NbConvertApp] Converting notebook 04 - Custom Display Logic.ipynb to html
[NbConvertApp] Support files will be in 04 - Custom Display Logic_files/
[NbConvertApp] Loaded template html_full.tpl
[NbConvertApp] Writing 221081 bytes to 04 - Custom Display Logic.html


Html is the default value (that can be configured) , so the verbose form would be 

In [2]:
%%bash
ipython nbconvert  --to=html '04 - Custom Display Logic.ipynb'

[NbConvertApp] Using existing profile dir: u'/Users/bussonniermatthias/.ipython/profile_default'
[NbConvertApp] Converting notebook 04 - Custom Display Logic.ipynb to html
[NbConvertApp] Support files will be in 04 - Custom Display Logic_files/
[NbConvertApp] Loaded template html_full.tpl
[NbConvertApp] Writing 221081 bytes to 04 - Custom Display Logic.html


You can also convert to latex, which will take care of extractin the embeded base64 encoded png, or the svg and call inkscape to convert those svg to pdf if necessary :

In [3]:
%%bash
ipython nbconvert  --to=latex '04 - Custom Display Logic.ipynb'

[NbConvertApp] Using existing profile dir: u'/Users/bussonniermatthias/.ipython/profile_default'
[NbConvertApp] Converting notebook 04 - Custom Display Logic.ipynb to latex
[NbConvertApp] Support files will be in 04 - Custom Display Logic_files/
Setting Language: .UTF-8

(process:26432): Gtk-WARNING **: Locale not supported by C library.
	Using the fallback 'C' locale.
Setting Language: .UTF-8

(process:26472): Gtk-WARNING **: Locale not supported by C library.
	Using the fallback 'C' locale.
Setting Language: .UTF-8

(process:26512): Gtk-WARNING **: Locale not supported by C library.
	Using the fallback 'C' locale.
Setting Language: .UTF-8

(process:26552): Gtk-WARNING **: Locale not supported by C library.
	Using the fallback 'C' locale.
Setting Language: .UTF-8

(process:26592): Gtk-WARNING **: Locale not supported by C library.
	Using the fallback 'C' locale.
[NbConvertApp] Loaded template latex_article.tplx
[NbConvertApp] Writing 41196 bytes to 04 - Custom Display Logic.tex


You should just have to compile the generated `.tex` file. If you get the required packages installed, if should compile out of the box.

For convenience we allow to run extra action after the conversion has been done, in particular for `latex` we have a `pdf` post-processor. 
You can define the postprocessor tu run with the `--post` flag.

In [4]:
%%bash
ipython nbconvert  --to=latex '04 - Custom Display Logic.ipynb' --post=pdf

[NbConvertApp] Using existing profile dir: u'/Users/bussonniermatthias/.ipython/profile_default'
[NbConvertApp] Converting notebook 04 - Custom Display Logic.ipynb to latex
[NbConvertApp] Support files will be in 04 - Custom Display Logic_files/
Setting Language: .UTF-8

(process:26658): Gtk-WARNING **: Locale not supported by C library.
	Using the fallback 'C' locale.
Setting Language: .UTF-8

(process:26698): Gtk-WARNING **: Locale not supported by C library.
	Using the fallback 'C' locale.
Setting Language: .UTF-8

(process:26738): Gtk-WARNING **: Locale not supported by C library.
	Using the fallback 'C' locale.
Setting Language: .UTF-8

(process:26778): Gtk-WARNING **: Locale not supported by C library.
	Using the fallback 'C' locale.
Setting Language: .UTF-8

(process:26818): Gtk-WARNING **: Locale not supported by C library.
	Using the fallback 'C' locale.
[NbConvertApp] Loaded template latex_article.tplx
[NbConvertApp] Writing 41196 bytes to 04 - Custom Display Logic.tex
[NbCon

Have a look at `04 - Custom Display Logic.pdf`, toward the end, where we compared `display()` vs `display_html()` and returning the object.
See how the cell where we use `display_html` was not able to display the circle, whereas the two other ones were able to select one of the oher representation they know how to display.

### Customizing template

let's look at the first 20 lines of the `python` exporter

In [5]:
pyfile = !ipython nbconvert --to python '04 - Custom Display Logic.ipynb' --stdout
for l in pyfile[20:40]:
    print l

# 1. Implementing special display methods such as `_repr_html_`.
# 2. Registering a display function for a particular type.
# 
# In this Notebook we show how both approaches work.

# Before we get started, we will import the various display functions for displaying the different formats we will create.

# In[54]:

from IPython.display import display
from IPython.display import (
    display_html, display_jpeg, display_png,
    display_javascript, display_svg, display_latex
)


### Implementing special display methods

# The main idea of the first approach is that you have to implement special display methods, one for each representation you want to use. Here is a list of the names of the special methods and the values they must return:
# 


We see that the non-code cell are exported to the file. To have a cleaner script, we will export only the code contained in the code cells.

To do so, we will inherit the python template, and overwrite the markdown blocks to be empty.

In [6]:
%%writefile simplepython.tpl
{% extends 'python.tpl'%}

{% block markdowncell -%}
{% endblock markdowncell %}

## we also want to get rig of header cell
{% block headingcell -%}
{% endblock headingcell %}

## and let's change the appearance of input prompt
{% block in_prompt %}
# This was input cell with prompt number : {{ cell.prompt_number if cell.prompt_number else ' ' }}
{%- endblock in_prompt %}

Overwriting simplepython.tpl


In [7]:
pyfile = !ipython nbconvert --to python '04 - Custom Display Logic.ipynb' --stdout --template=simplepython.tpl

for l in pyfile[4:40]:
    print l
print '...'


# This was input cell with prompt number : 54
from IPython.display import display
from IPython.display import (
    display_html, display_jpeg, display_png,
    display_javascript, display_svg, display_latex
)


# This was input cell with prompt number : 55
get_ipython().magic(u'load soln/mycircle.py')


# This was input cell with prompt number : 56
class MyCircle(object):
    
    def _repr_html_(self):
        return "&#x25CB; (<b>html</b>)"

    def _repr_svg_(self):
        return """<svg width='100px' height='100px'>
           <circle cx="50" cy="50" r="20" stroke="black" stroke-width="1" fill="blue"/>
        </svg>"""
    
    def _repr_latex_(self):
        return r"$\bigcirc \LaTeX$"

    def _repr_javascript_(self):
        return "alert('I am a circle!');"


# This was input cell with prompt number : 57
c = MyCircle()


# This was input cell with prompt number : 58
...


I'll let you read Jinja manual for the exact syntax of the template.

## Template that use cells metadata

Notebook fileformat support attaching arbitrary JSON metadata to each cell of a notebook. In this part we will use those metadata.

First you need to choose another notebook you want to convert to html, and tag some of the cell with metadata.
You can see the file `soln/celldiff.js` for a solution on how to tag, or follow the javascript tutorial to see how to do that. Use what we have written there to tag cells of some notebooks to `Easy`|`Medium`|`Hard`|`<None>`, and convert this notebook using your template.

you might need the following : 
```
{% extends 'html_full.tpl'%}
{% block any_cell %}
{{ super() }}
<div style="background-color:red">
<div style='background-color:orange'>
```

`metadata` might not exist, be sure to :

`cell['metadata'].get('example',{}).get('difficulty','')`

tip: use `%%writefile` to edit the template in the notebook :-)

In [8]:
%%bash
# ipython nbconvert --to html <your chosen notebook.ipynb> --template=<your template file>

In [ ]:
%loadpy soln/coloreddiff.tpl

In [ ]:
# ipython nbconvert --to html '04 - Custom Display Logic.ipynb' --template=soln/coloreddiff.tpl

### Get rid of all command line flags.

As of all of IPython nbconvert can be configured using profiles and passing the `--profile` flag. 
Moreover if a `config.py` file exist in current working directory nbconvert will use that, or read the config file you give to it with the `--config=<file>` flag. 

In the end, if you are often running nbconvert on the sam project, `$ ipython nbconvert` should be enough to get you up and ready.